In [1]:
import os
import sys
from dotenv import find_dotenv

In [2]:
path_to_utils_module = os.path.join(os.path.dirname(find_dotenv()), '..', 'integration_tests', 'tests')
sys.path.insert(0, path_to_utils_module)

In [3]:
import flowclient
from datetime import timedelta
from utils import make_token

In [4]:
claims = {
    'daily_location': {'permissions': ['run', 'poll', 'get_result'], 'spatial_aggregation': ['admin3', 'admin2']},
    'modal_location': {'permissions': ['run', 'poll', 'get_result'], 'spatial_aggregation': ['admin3', 'admin2']},
    'flows': {'permissions': ['run', 'poll', 'get_result'], 'spatial_aggregation': ['admin3', 'admin2', 'admin1']},
}

In [5]:
TOKEN = make_token(username='testuser', secret_key='secret', lifetime=timedelta(days=1), claims=claims)

In [6]:
conn = flowclient.connect("http://localhost:9090", TOKEN)

/Users/maxalbert/work/code/flowminder/flowkit/flowclient/flowclient/client.py:159: UserWarning: Communications with this server are NOT SECURE.
  return Connection(url, token, api_version, ssl_certificate)


## Daily location

In [7]:
query_spec = flowclient.daily_location(date="2016-01-01", aggregation_unit="admin3", daily_location_method="last", subscriber_subset=None)

In [8]:
query_id = flowclient.run_query(conn, query_spec)
print(query_id)

4a12fae60ce647d01b2bd5902b8a48e7


In [9]:
flowclient.query_is_ready(conn, query_id)

(False, <Response [202]>)

In [10]:
#print(flowclient.get_status(conn, query_id))

In [11]:
#res = flowclient.get_result(conn, query_id)
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,name,total
0,Dolpa,43
1,Bajhang,17
2,Kavrepalanchok,14
3,Rolpa,12
4,Rukum,14


Alternative:

In [12]:
df = flowclient.get_result(conn, query_spec)
df.head()

,name,total
0,Dolpa,43
1,Bajhang,17
2,Kavrepalanchok,14
3,Rolpa,12
4,Rukum,14


## Modal location

In [13]:
dl1 = flowclient.daily_location(date="2016-01-01", aggregation_unit="admin3", daily_location_method="last", subscriber_subset=None)
dl2 = flowclient.daily_location(date="2016-01-02", aggregation_unit="admin3", daily_location_method="last", subscriber_subset=None)

In [14]:
#ml = flowclient.modal_location(dl1, dl2, aggregation_unit="admin3")
ml = flowclient.modal_location_from_dates("2016-01-01", "2016-01-03", aggregation_unit="admin3", daily_location_method="last", subscriber_subset=None)

In [15]:
ml

{'query_kind': 'modal_location',
 'params': {'locations': ({'query_kind': 'daily_location',
    'params': {'date': '2016-01-01',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}},
   {'query_kind': 'daily_location',
    'params': {'date': '2016-01-02',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}},
   {'query_kind': 'daily_location',
    'params': {'date': '2016-01-03',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}}),
  'aggregation_unit': 'admin3'}}

In [16]:
query_id = flowclient.run_query(conn, ml)
print(query_id)

64148ac6a93121d8950ea1e70faaf4c8


In [17]:
flowclient.query_is_ready(conn, query_id)

(False, <Response [202]>)

In [18]:
#res = flowclient.get_result(conn, query_id)
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,name,total
0,Dolpa,65
1,Bajhang,13
2,Kavrepalanchok,27
3,Rolpa,17
4,Rukum,12


In [19]:
df = flowclient.get_result(conn, ml)
df.head()

,name,total
0,Dolpa,65
1,Bajhang,13
2,Kavrepalanchok,27
3,Rolpa,17
4,Rukum,12


## Flow

In [20]:
aggregation_unit = "admin1"
ml = flowclient.modal_location_from_dates("2016-01-01", "2016-01-04", aggregation_unit=aggregation_unit, daily_location_method="last", subscriber_subset=None)
dl = flowclient.daily_location("2016-01-07", aggregation_unit=aggregation_unit, daily_location_method="last", subscriber_subset=None)

In [21]:
flow = flowclient.flows(ml, dl, aggregation_unit=aggregation_unit)

In [22]:
query_id = flowclient.run_query(conn, flow)
print(query_id)

35edc41fc37f7630f5d96fa2ed3830e0


In [23]:
flowclient.query_is_ready(conn, query_id)

(False, <Response [202]>)

In [24]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,count,name_from,name_to
0,7,Central Development Region,Western Development Region
1,10,Central Development Region,Mid-Western Development Region
2,5,Central Development Region,Far-Western Development Region
3,2,Central Development Region,Eastern Development Region
4,3,Central Development Region,Central Development Region
